# Joining and Appending DataFrames in PySpark HW

Now it's time to test your knowledge and further engrain the concepts we touched on in the lectures. Let's go ahead and get started.




**As always let's start our Spark instance.**

In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import os

import findspark
findspark.init()
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *#avg, count, expr
from pyspark.sql.types import *

In [4]:
# initialize
sc = pyspark.SparkContext()
spark = SparkSession(sc)
spark.sparkContext.appName = 'joinAppend'
# show the number of cores
print('%d cores'%spark._jsc.sc().getExecutorMemoryStatus().keySet().size())
spark

1 cores


## Read in the database

Let cotinue working with our college courses dataframe to get some more insights and practice what we have learned!Let's read in the whole database using the loop function that we learned about in the lecture to automatically read in all the datasets from the uw-madision-courses folder (there are too many datasets to each one individually.

In [5]:
# load univ of wisconsin madison data into a dict of dataframes
path = '../../data/uw-madison-courses/'
uofw = {f.split('.')[0]:spark.read.format('csv').options(header=True, inferSchema=True).load(path+f) for f in os.listdir(path) if f[-4:] == '.csv'}

Now check the contents of a few of the dataframses that were read in above.

In [6]:
# show the head of each
for (key, val) in uofw.items():
    print(key)
    val.show(5, truncate=False)

subject_memberships
+------------+------------------------------------+
|subject_code|course_offering_uuid                |
+------------+------------------------------------+
|220         |344b3ebe-da7e-314c-83ed-9425269695fd|
|320         |344b3ebe-da7e-314c-83ed-9425269695fd|
|346         |344b3ebe-da7e-314c-83ed-9425269695fd|
|612         |344b3ebe-da7e-314c-83ed-9425269695fd|
|636         |344b3ebe-da7e-314c-83ed-9425269695fd|
+------------+------------------------------------+
only showing top 5 rows

instructors
+-------+------------------+
|id     |name              |
+-------+------------------+
|761703 |JOHN ARCHAMBAULT  |
|3677061|STEPHANIE KANN    |
|788586 |KATHY PREM        |
|1600463|KRISTIN KLARKOWSKI|
|693634 |DAVID BOHNHOFF    |
+-------+------------------+
only showing top 5 rows

rooms
+------------------------------------+-------------+---------+
|uuid                                |facility_code|room_code|
+------------------------------------+-------------+-----

## Recap: About this database

You will notice that there are several more tables in the uw-madision-courses folder than there are read in above. This so that you will have a chance to practice your own custom joins and learn about the relationships between a real database work. Sometimes we don't know how they are related and we need to figure it out! I'll save that for the HW :) 

Here is a look at some of the important variables we can use to join our tables:

 - course_offerings: uuid, course_uuid, term_code, name
 - instructors: id, name
 - schedules: uuid
 - sections: uuid, course_offering_uuid,room_uuid, schedule_uuid
 - teachings: instructor_id, section_uuid
 - courses: uuid
 - grade_distributions: course_offering_uuid,section_number
 - rooms: uuid, facility_code, room_code
 - subjects: code
 - subject_memberships: subject_code, course_offering_uuid
 
 **Source:** https://www.kaggle.com/Madgrades/uw-madison-courses
 
So alright, let's use this information to discover some insights from this data!

## 1a. Can you assign the room numbers to each section of each course?

Show only the rooms uuid, facility code, room number, term code and the name of the course from the course_offerings table.

In [ ]:
joined = uofw['course_offerings'].withColumnRenamed('uuid', 'course_offering_uuid').withColumnRenamed('name', 'course_name')\
    .join(uofw['sections'].withColumnRenamed('uuid', 'section_uuid'), how='inner', on='course_offering_uuid')\
    .join(uofw['rooms'].withColumnRenamed('uuid', 'room_uuid'), how='inner', on='room_uuid')

display(joined.select('course_name', 'term_code', 'facility_code', 'room_code', 'room_uuid').limit(4).toPandas())

## 1b. Now show same output as above but for only facility number 0469 (facility_code)

In [ ]:
joined.select('course_name', 'term_code', 'facility_code', 'room_code', 'room_uuid').where(col('facility_code')=='0469').show(truncate=False)

## 2. Count how many sections are offered for each subject for each facility

*Note: this will involve a groupby*

In [ ]:
joined.join(uofw['subject_memberships'], how='inner', on='course_offering_uuid')\
    .groupBy('subject_code', 'facility_code').count().orderBy(col('count').desc()).show()

## 3. What are the hardest classes?

Let's see if we can figure out which classes are the hardest by seeing how many students failed. Note that you will first need to aggregate the grades table by the course uuid to include all sections. Show the name of the course as well that you will need to get from the course_offering table.

In [7]:
hardest = uofw['grade_distributions'].select('course_offering_uuid', 'f_count', (col('a_count') + col('ab_count') + col('b_count') + col('bc_count') +
                                                                                 col('c_count') + col('d_count') + col('f_count')).alias('students'))\
    .groupBy('course_offering_uuid').agg(sum('f_count').alias('total_failed'), sum('students').alias('total_students'))\
    .join(uofw['course_offerings'].withColumnRenamed('uuid', 'course_offering_uuid').withColumnRenamed('name', 'course_name'))\
    .select('course_name', 'total_students', 'total_failed', (100*col('total_failed')/col('total_students')).alias('perc_failed'))\
    .orderBy(col('perc_failed').desc())

In [11]:
hardest.where(col('total_students') > 10).show(10, truncate=False)

+--------------------------+--------------+------------+-----------------+
|course_name               |total_students|total_failed|perc_failed      |
+--------------------------+--------------+------------+-----------------+
|Cooperative Education Prog|26            |8           |30.76923076923077|
|Cooperative Education Prog|26            |8           |30.76923076923077|
|Cooperative Education Prog|26            |8           |30.76923076923077|
|Cooperative Education Prog|26            |8           |30.76923076923077|
|Cooperative Education Prog|26            |8           |30.76923076923077|
|Cooperative Education Prog|26            |8           |30.76923076923077|
|Cooperative Education Prog|26            |8           |30.76923076923077|
|Cooperative Education Prog|26            |8           |30.76923076923077|
|Cooperative Education Prog|26            |8           |30.76923076923077|
|Cooperative Education Prog|26            |8           |30.76923076923077|
+------------------------

## Challenge Question: Automating data entry errors

We see in the dataframe below that there are several typos of various animal names. If this was a large database of several millions of records, correcting these errors would be way too labor intensive. How can we automate correcting these errors?

*Hint: Leven...*

In [ ]:
values = [('Monkey',10),('Monkay',36),('Mnky',123), \
          ('Elephant',48),('Elefant',16),('Ellafant',1), \
          ('Hippopotamus',48),('Hipopotamus',16),('Hippo',1)]
zoo = spark.createDataFrame(values,['Animal','age'])
zoo.show()

In [ ]:
# not really interested in this...

### Great job! 

In [12]:
sc.stop()